In [1]:
import numpy as np

def x(X):
    sample = np.random.random([512])
    sampleX = np.stack([sample[x:x+24] for x in range(511-24)])
    sampleY = np.stack([sample[x+24] for x in range(511-24)])
    np.linalg.pinv((sampleX.T @ sampleX)) @ sampleX.T @ sampleY

In [2]:
import numpy as np
import scipy.io.wavfile as sio
import os

In [3]:
ROOT = "./datasets/clarin-long/data"

In [4]:
def get_fnames_from_clarin_corpus(ROOT):
    rec_fnames, trans_fnames = [], []
    for i in [x for x in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, x))]:
        recordings = os.path.join(ROOT, i, "wav")
        transcripts = os.path.join(ROOT, i, "lab")
        for fname in os.listdir(recordings):
            core, extension = fname.split(".")
            assert extension == "wav"
            if os.path.isfile(os.path.join(transcripts, core + ".hlb")):
                rec_fnames.append(os.path.join(recordings, fname))
                trans_fnames.append(os.path.join(transcripts, core + ".hlb"))
    return rec_fnames, trans_fnames

rec_fnames, trans_fnames = get_fnames_from_clarin_corpus(ROOT)

In [5]:
def get_recording_lengths(fnames):
    lens = []
    for f in fnames:
        sr, data = sio.read(f)
        assert sr == 16000
        assert len(data.shape) == 1
        lens.append(len(data))
    return lens

lens = get_recording_lengths(rec_fnames)

In [6]:
sorted_lens = np.array(sorted(lens))
step = len(sorted_lens) // 10
delimiter_lens = [sorted_lens[step * i] for i in range(10)]
delimiters = [step * i for i in range(11)]

In [7]:
delimiter_lens.append(sorted_lens[-1] + 128)
indices = []
for i in range(10):
    lb, hb = delimiter_lens[i], delimiter_lens[i+1]
    indices.append((lb <= np.array(lens)) * (np.array(lens) < hb))

In [8]:
delimiter_lens

[64269,
 141920,
 166560,
 186240,
 205120,
 223520,
 243600,
 266000,
 293600,
 338880,
 1585088]

In [9]:
def get_phones_clarin(fname):
    with open(fname, "r", encoding="utf-8") as f:
        s = f.read()
        s = s.split("Phoneme Phoneme")[1]
        s = s.split("\n\n")[0]
        s = [x.split(' ')[1] for x in s.split('\n') if x.strip()]
        s = [x.split('_')[0] for x in s]
    return s

transes = (get_phones_clarin(x) for x in trans_fnames)
all_phones = set()

def len_add_set(trans):
    global all_phones
    all_phones |= set(trans)
    return len(trans)

trans_lens = [len_add_set(x) for x in transes]
list(all_phones), max(trans_lens), len(list(all_phones))

(['tS',
  'dz',
  'sil',
  'en',
  'e',
  'I',
  'v',
  'S',
  'i',
  'r',
  'dzi',
  'on',
  'f',
  'n',
  'si',
  'o',
  't',
  'k',
  'l',
  'ts',
  's',
  'x',
  'z',
  'ni',
  'b',
  'j',
  'zi',
  'g',
  'p',
  'dZ',
  'Z',
  'tsi',
  'a',
  'd',
  'm',
  'w',
  'u'],
 423,
 37)

In [20]:
import librosa
import scipy

NFEAT = 24

phone_dict = list(all_phones)
phone_zero = len(phone_dict)

def pcen(S, sr=22050, hop_length=512, gain=0.98, bias=2, power=0.5,
         time_constant=0.400, eps=1e-6, b=None, max_size=1, ref=None,
         axis=-1, max_axis=None):
    """
    Adapted from librosa
    """
    if b is None:
        t_frames = time_constant * sr / float(hop_length)
        # By default, this solves the equation for b:
        #   b**2  + (1 - b) / t_frames  - 2 = 0
        # which approximates the full-width half-max of the
        # squared frequency response of the IIR low-pass filter
        b = (np.sqrt(1 + 4 * t_frames**2) - 1) / (2 * t_frames**2)

    if np.issubdtype(S.dtype, np.complexfloating):
        S = np.abs(S)

    if ref is None:
        if max_size == 1:
            ref = S
        elif S.ndim == 1:
            raise ParameterError('Max-filtering cannot be applied to 1-dimensional input')
        else:
            if max_axis is None:
                if S.ndim != 2:
                    raise ParameterError('Max-filtering a {:d}-dimensional spectrogram '
                                         'requires you to specify max_axis'.format(S.ndim))
                # if axis = 0, max_axis=1
                # if axis = +- 1, max_axis = 0
                max_axis = np.mod(1 - axis, 2)

            ref = scipy.ndimage.maximum_filter1d(S, max_size, axis=max_axis)

    S_smooth = scipy.signal.lfilter([b], [1, b - 1], ref, axis=axis)

    # Working in log-space gives us some stability, and a slight speedup
    smooth = np.exp(-gain * (np.log(eps) + np.log1p(S_smooth / eps)))
    return (S * smooth + bias)**power - bias**power

def callback(data):
    spec = librosa.stft(data, n_fft=512, hop_length=128)
    spec = pcen(spec, sr=16000, hop_length=128)
    return librosa.feature.mfcc(S=spec, sr=16000, hop_length=128, n_mfcc=24).T

# callback = lambda x: np.log(np.abs(librosa.stft(x, n_fft=512, hop_length=128).T) ** 2) + 2e-12)
# callback = lambda x: librosa.feature.mfcc(S=librosa.feature.melspectrogram(x, sr=16000, n_fft=512, hop_length=128), sr=16000).T

# N_SIZE = 1
# BIN_SIZE = 257
# BIN_SIZE = 20
LENGTHS = [1 + (x - 512) // 128 for x in delimiter_lens[1:]]

for ix, stratum in enumerate(indices):
    ixes = np.where(stratum)[0]
    shape = [len(ixes), LENGTHS[ix] + 4, NFEAT]
    print(shape)
    phones_shape = [len(ixes), max([trans_lens[x] for x in ixes]) + 1]
    specs, transes = None, None
    specs = np.zeros(shape, np.float32)
    transes = np.ones(phones_shape, np.uint16) * phone_zero
    for num, rec_ix in enumerate(ixes):
        print(num)
        fname = rec_fnames[rec_ix]
        data = sio.read(fname)[1].astype(np.float32) / 2**15
        stft = callback(data)
        # print(stft.shape, specs.shape)
        specs[num, :stft.shape[0], :NFEAT] = stft
        trans = get_phones_clarin(trans_fnames[rec_ix])
        trans = np.array([phone_dict.index(x) for x in trans])
        transes[num, :len(trans)] = trans
    np.save(os.path.join(ROOT, "clarin-mfcc-rec-aligned-PCEN-MFCC-{}".format(ix)), specs)
    np.save(os.path.join(ROOT, "clarin-mfcc-trans-aligned-PCEN-MFCC-{}".format(ix)), transes)
    print("Saved batch", ix)

[1377, 1109, 24]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
2

628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877


1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376


356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605


954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1

85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338

684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933


1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
Saved batch 9


In [15]:
import gc
gc.collect()

175

In [ ]:
import librosa
librosa.feature.mfcc

In [19]:
librosa.feature.mfcc?

In [ ]:
import numpy as np

In [ ]:
time = np.random.normal(size=[32000,])

In [ ]:
librosa.feature.mfcc(time, sr=16000).shape

In [ ]:
librosa.stft(time, n_fft=512)

In [ ]:
librosa.feature.mfcc(S=librosa.feature.melspectrogram(time, sr=16000, n_fft=512, hop_length=128), sr=16000).shape

In [ ]:
with open("/pictec/datasets/clarin-long/data/SES0001/lab/sent001.plb", encoding="utf-8") as f:
    text = f.read()

In [ ]:
print(text)

In [11]:
delimiter_lens[1:]

[141920,
 166560,
 186240,
 205120,
 223520,
 243600,
 266000,
 293600,
 338880,
 1585088]